In [ ]:
from __future__ import annotations

%matplotlib inline
import warnings

warnings.filterwarnings('ignore')

# Seasonal trends with the GOZCARDS Dataset

Here we calculate seasonal trends using the GOZCARDS dataset by regressing to the VMR monthly zonal means using seasonal terms in our predictors.

In [ ]:
import numpy as np
import xarray as xr

from LOTUS_regression.predictors import load_data
from LOTUS_regression.predictors.seasonal import add_seasonal_components
from LOTUS_regression.regression import regress_all_bins

The GOZCARDS data is in multiple NetCDF4 files.  Load them all in and concatenate on the time dimension.  Also only take values in the latitude range -60 to 60.

In [ ]:
GOZCARDS_FILES = r'/home/runner/work/lotus-regression/lotus-regression/test_data/GOZCARDS/*.nc4'

data = xr.decode_cf(xr.open_mfdataset(GOZCARDS_FILES, combine='nested', concat_dim='time', group='Merged', engine='netcdf4'))

data = data.loc[dict(lat=slice(-60, 60))]

print(data)

Load some standard predictors and add a constant

In [ ]:
predictors = load_data('pred_baseline_pwlt.csv')

print(predictors.columns)

Currently our predictors have no seasonal dependence.  Add in some seasonal terms with different numbers of Fourier components.

In [ ]:
predictors = add_seasonal_components(predictors, {'constant': 4, 'linear_pre': 4, 'linear_post': 4, 'qboA': 2, 'qboB': 2})

print(predictors[:5])

Perform the regression and convert the coefficients to percent anomaly.  We pass include_monthly_fits = True so that
the seasonal fits are used to calculate monthly trends.  The results at the end will include 'linear_post_monthly'
and 'linear_post_monthly_std' that are the monthly trend terms and errors respectively

In [ ]:
data = data.sel(lev=slice(100, 0.2)).sel(lat=slice(-65, 65))   # remove bins without data
results = regress_all_bins(predictors, data['average'], tolerance=0.1, include_monthly_fits=True)

# Convert to ~ percent
results /= data['average'].mean(dim='time')
results *= 100

print(results)

In [ ]:
import matplotlib.pyplot as plt

import LOTUS_regression.plotting.trends as trends

# Plot the seasonal post trends at the level closest to 2 hPa (2.15 hPa)
trends.plot_with_confidence(results.sel(lev=2, method='nearest'), 'linear_post_monthly', x='lat', y='month')
plt.xlabel('Latitude')
plt.ylabel('Month')